In [2]:
from bs4 import BeautifulSoup
from collections import defaultdict
from collections import Counter
import os
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import math
import csv
import re



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Prometheus\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
path = "D:/Degree of Master/code/CSCI 5523/Project 2/reuters21578"
files = os.listdir(path)
filesToOpen = [] 
for file in files:
    if os.path.splitext(file)[1] ==".sgm" :
        filePath = path + '/' + file
        filesToOpen.append(filePath) # find all 22 sgm files

In [4]:
topicCount = defaultdict(int)
for filePath in filesToOpen:
    with open(filePath,'rb') as f:
        soup = BeautifulSoup(f)  
    for topicTag in soup.find_all("topics"):
       # print(topicTag)
        if len(list(topicTag.children)) == 1 :
            tag = next(topicTag.children)
            topicStr = str(tag.contents[0])
            topicCount[topicStr] += 1

In [5]:
sum = 0
for i in topicCount.values():
    sum+=i
sum # of qualifying articles

9494

In [6]:
d = Counter(topicCount)
mostCommonTopics = d.most_common(20) # find 20 most common topicsmostCommonTopics
mostCommonTopics

[('earn', 3945),
 ('acq', 2362),
 ('crude', 408),
 ('trade', 361),
 ('money-fx', 307),
 ('interest', 285),
 ('money-supply', 161),
 ('ship', 158),
 ('sugar', 143),
 ('coffee', 116),
 ('gold', 99),
 ('gnp', 83),
 ('cpi', 79),
 ('cocoa', 63),
 ('jobs', 55),
 ('copper', 54),
 ('reserves', 53),
 ('grain', 51),
 ('alum', 50),
 ('ipi', 49)]

In [7]:
# Selecting the Subset of the Dataset for Clustering
labelList = []
bodyDict = defaultdict(str)
for filePath in filesToOpen:
    with open(filePath,'rb') as f:
        soup = BeautifulSoup(f,"html.parser")  
    for topicTag in soup.find_all("topics"):
       # print(topicTag)
        if len(list(topicTag.children)) == 1:
            tag = next(topicTag.children)
            topicStr = str(tag.contents[0])
            for topicFrequencyTuple in mostCommonTopics:
                if topicStr == topicFrequencyTuple[0]:
                    newid = int(topicTag.parent['newid'])
                    labelList.append((newid,topicStr))       #build i,label dict
                    if type(topicTag.parent.body) == type(tag) : 
                        bodyDict[newid] = topicTag.parent.body.contents[0]   #bulid i,article dict
bodyDict                

defaultdict(str,
            {1: 'Showers continued throughout the week in\nthe Bahia cocoa zone, alleviating the drought since early\nJanuary and improving prospects for the coming temporao,\nalthough normal humidity levels have not been restored,\nComissaria Smith said in its weekly review.\n    The dry period means the temporao will be late this year.\n    Arrivals for the week ended February 22 were 155,221 bags\nof 60 kilos making a cumulative total for the season of 5.93\nmln against 5.81 at the same stage last year. Again it seems\nthat cocoa delivered earlier on consignment was included in the\narrivals figures.\n    Comissaria Smith said there is still some doubt as to how\nmuch old crop cocoa is still available as harvesting has\npractically come to an end. With total Bahia crop estimates\naround 6.4 mln bags and sales standing at almost 6.2 mln there\nare a few hundred thousand bags still in the hands of farmers,\nmiddlemen, exporters and processors.\n    There are doubts as

In [57]:
# Obtaining the Sparse Representation
stoplistPath = "D:/Degree of Master/code/CSCI 5523/Project 2/stoplist.txt"
porter = PorterStemmer()
originalInput = []
with open(stoplistPath,'r') as stopf:
        stoplist = stopf.read() 
stopList = word_tokenize(str(stoplist))
tokenDict = defaultdict(int)
for newid in bodyDict.keys():
    tokenCounterByID = defaultdict(int)
    re.sub(r'[^\x00-\x7f]',r' ',bodyDict[newid]) #step 1
    bodyDict[newid].replace("&lt","<")
    bodyDict[newid].replace("&ge",">") #step 2
    bodyDict[newid].lower() #step 3
    for c in bodyDict[newid]:
        if not c.isalnum() : bodyDict[newid] = bodyDict[newid].replace(c," ") #step 4
    tokens = word_tokenize(bodyDict[newid]) #step 5
    tokens = [token for token in tokens if not token.isnumeric()] #step 6
    tokens = [token for token in tokens if token not in stopList] #step 7
    tokens = [porter.stem(token) for token in tokens] #step 8
    for token in tokens:
        tokenCounterByID[token] +=1
        tokenDict[token] += 1
    for token in tokenCounterByID.keys():
        originalInput.append((newid,token,tokenCounterByID[token]))
originalInput = [(newid,token,freq) for newid,token,freq in originalInput if tokenDict[token] >= 5] #step 9        
originalInput
    
          

[(1, 'shower', 1),
 (1, 'continu', 1),
 (1, 'week', 3),
 (1, 'bahia', 4),
 (1, 'cocoa', 6),
 (1, 'zone', 1),
 (1, 'allevi', 1),
 (1, 'drought', 1),
 (1, 'januari', 1),
 (1, 'improv', 1),
 (1, 'prospect', 1),
 (1, 'come', 1),
 (1, 'temporao', 2),
 (1, 'normal', 1),
 (1, 'level', 1),
 (1, 'restor', 1),
 (1, 'comissaria', 5),
 (1, 'smith', 5),
 (1, 'weekli', 1),
 (1, 'review', 1),
 (1, 'the', 1),
 (1, 'dri', 1),
 (1, 'period', 2),
 (1, 'mean', 1),
 (1, 'late', 1),
 (1, 'arriv', 2),
 (1, 'februari', 3),
 (1, 'bag', 5),
 (1, 'kilo', 2),
 (1, 'cumul', 1),
 (1, 'total', 3),
 (1, 'season', 1),
 (1, 'mln', 5),
 (1, 'stage', 1),
 (1, 'deliv', 1),
 (1, 'earlier', 1),
 (1, 'includ', 1),
 (1, 'figur', 2),
 (1, 'doubt', 2),
 (1, 'crop', 5),
 (1, 'harvest', 1),
 (1, 'practic', 1),
 (1, 'with', 1),
 (1, 'estim', 2),
 (1, 'sale', 7),
 (1, 'stand', 1),
 (1, 'hundr', 1),
 (1, 'thousand', 1),
 (1, 'hand', 1),
 (1, 'farmer', 2),
 (1, 'export', 2),
 (1, 'processor', 1),
 (1, 'there', 1),
 (1, 'fit', 1),
 (1

In [58]:
# Vector Representation
tokenDimensionNum = defaultdict(int)
i = 0
for token in tokenDict.keys():
    if tokenDict[token] >= 5:
        tokenDimensionNum[token] = i
        i+=1
originalInput = [(newid,tokenDimensionNum[token],freq) for newid,token,freq in originalInput]
originalInput

[(1, 0, 1),
 (1, 1, 1),
 (1, 2, 3),
 (1, 3, 4),
 (1, 4, 6),
 (1, 5, 1),
 (1, 6, 1),
 (1, 7, 1),
 (1, 8, 1),
 (1, 9, 1),
 (1, 10, 1),
 (1, 11, 1),
 (1, 12, 2),
 (1, 13, 1),
 (1, 14, 1),
 (1, 15, 1),
 (1, 16, 5),
 (1, 17, 5),
 (1, 18, 1),
 (1, 19, 1),
 (1, 20, 1),
 (1, 21, 1),
 (1, 22, 2),
 (1, 23, 1),
 (1, 24, 1),
 (1, 25, 2),
 (1, 26, 3),
 (1, 27, 5),
 (1, 28, 2),
 (1, 29, 1),
 (1, 30, 3),
 (1, 31, 1),
 (1, 32, 5),
 (1, 33, 1),
 (1, 34, 1),
 (1, 35, 1),
 (1, 36, 1),
 (1, 37, 2),
 (1, 38, 2),
 (1, 39, 5),
 (1, 40, 1),
 (1, 41, 1),
 (1, 42, 1),
 (1, 43, 2),
 (1, 44, 7),
 (1, 45, 1),
 (1, 46, 1),
 (1, 47, 1),
 (1, 48, 1),
 (1, 49, 2),
 (1, 50, 2),
 (1, 51, 1),
 (1, 52, 1),
 (1, 53, 1),
 (1, 54, 2),
 (1, 55, 1),
 (1, 56, 1),
 (1, 57, 1),
 (1, 58, 1),
 (1, 59, 1),
 (1, 60, 1),
 (1, 61, 1),
 (1, 62, 1),
 (1, 63, 1),
 (1, 64, 2),
 (1, 65, 1),
 (1, 66, 1),
 (1, 67, 2),
 (1, 68, 1),
 (1, 69, 1),
 (1, 70, 1),
 (1, 71, 1),
 (1, 72, 1),
 (1, 73, 2),
 (1, 74, 2),
 (1, 75, 1),
 (1, 76, 4),
 (1, 77, 

In [59]:
len(tokenDimensionNum)

5832

In [50]:
originalInput2 = originalInput
originalInput2 = [(newid,d,1+math.sqrt(freq)) for newid,d,freq in originalInput2]
originalInput2
originalInput3 = originalInput
originalInput3 = [(newid,d,1+math.log2(freq)) for newid,d,freq in originalInput2]
originalInput3 

[(1, 0, 2.0),
 (1, 1, 2.0),
 (1, 2, 2.449984313476496),
 (1, 3, 2.584962500721156),
 (1, 4, 2.7863829705865006),
 (1, 5, 2.0),
 (1, 6, 2.0),
 (1, 7, 2.0),
 (1, 8, 2.0),
 (1, 9, 2.0),
 (1, 10, 2.0),
 (1, 11, 2.271553303163612),
 (1, 12, 2.0),
 (1, 13, 2.0),
 (1, 14, 2.0),
 (1, 15, 2.6942419136306173),
 (1, 16, 2.6942419136306173),
 (1, 17, 2.0),
 (1, 18, 2.0),
 (1, 19, 2.0),
 (1, 20, 2.271553303163612),
 (1, 21, 2.0),
 (1, 22, 2.271553303163612),
 (1, 23, 2.449984313476496),
 (1, 24, 2.6942419136306173),
 (1, 25, 2.271553303163612),
 (1, 26, 2.0),
 (1, 27, 2.449984313476496),
 (1, 28, 2.0),
 (1, 29, 2.6942419136306173),
 (1, 30, 2.0),
 (1, 31, 2.0),
 (1, 32, 2.0),
 (1, 33, 2.0),
 (1, 34, 2.271553303163612),
 (1, 35, 2.271553303163612),
 (1, 36, 2.6942419136306173),
 (1, 37, 2.0),
 (1, 38, 2.0),
 (1, 39, 2.271553303163612),
 (1, 40, 2.866216153661624),
 (1, 41, 2.0),
 (1, 42, 2.0),
 (1, 43, 2.0),
 (1, 44, 2.0),
 (1, 45, 2.271553303163612),
 (1, 46, 2.271553303163612),
 (1, 47, 2.0),
 (1,

In [53]:
tokenDimensionNum

defaultdict(int,
            {'shower': 0,
             'continu': 1,
             'week': 2,
             'bahia': 3,
             'cocoa': 4,
             'zone': 5,
             'allevi': 6,
             'drought': 7,
             'januari': 8,
             'improv': 9,
             'prospect': 10,
             'temporao': 11,
             'normal': 12,
             'level': 13,
             'restor': 14,
             'comissaria': 15,
             'smith': 16,
             'weekli': 17,
             'review': 18,
             'dri': 19,
             'period': 20,
             'late': 21,
             'arriv': 22,
             'februari': 23,
             'bag': 24,
             'kilo': 25,
             'cumul': 26,
             'total': 27,
             'season': 28,
             'mln': 29,
             'stage': 30,
             'deliv': 31,
             'earlier': 32,
             'includ': 33,
             'figur': 34,
             'doubt': 35,
             'crop': 36,
          

In [11]:
k = len(tokenDimensionNum)
vectorPrototype = [0] * k
id = 0
id2 = 0
id3 = 0
vectorBody = defaultdict(list)
vectorBody2 = defaultdict(list)
vectorBody3 = defaultdict(list)
for tuple in originalInput:
    if id != tuple[0] and id == 0:
        id += 1
        vector = list(vectorPrototype)
    if id != tuple[0] and id > 0:
        vectorBody[id] = vector
        id += 1
        vector = list(vectorPrototype)
    vector[tuple[1]] = tuple[2]
for tuple in originalInput2:
    if id2 != tuple[0] and id2 == 0:
        id2 += 1
        vector = list(vectorPrototype)
    if id2 != tuple[0] and id2 > 0:
        vectorBody2[id2] = vector
        id2 += 1
        vector = list(vectorPrototype)
    vector[tuple[1]] = tuple[2]
for tuple in originalInput3:
    if id3 != tuple[0] and id3 == 0:
        id3 += 1
        vector = list(vectorPrototype)
    if id3 != tuple[0] and id3 > 0:
        vectorBody3[id3] = vector
        id3 += 1
        vector = list(vectorPrototype)
    vector[tuple[1]] = tuple[2]
vectorBody[1]

            
    

[1,
 1,
 3,
 4,
 6,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 0,
 1,
 1,
 5,
 5,
 1,
 1,
 1,
 2,
 1,
 1,
 2,
 3,
 5,
 2,
 1,
 3,
 1,
 5,
 1,
 1,
 1,
 0,
 1,
 2,
 2,
 5,
 1,
 1,
 2,
 7,
 1,
 1,
 1,
 1,
 2,
 0,
 2,
 1,
 1,
 2,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 2,
 1,
 1,
 1,
 0,
 1,
 1,
 2,
 2,
 1,
 4,
 14,
 2,
 3,
 1,
 1,
 3,
 5,
 8,
 4,
 5,
 1,
 1,
 0,
 4,
 7,
 3,
 6,
 1,
 0,
 2,
 2,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,

In [12]:
for df in vectorBody[3]:
    if df : print(df)

1


In [13]:
def normalize(vector):
    sum = 0.0
    for freq in vector:
        sum += math.pow(freq,2)
    length = math.sqrt(sum)
    for i in range(0,len(vector)):
        f = vector[i]/length
        vector[i] = f
    return vector

In [14]:
normalize(vectorBody[1])
vectorBody[1]

[0.034159349284257755,
 0.034159349284257755,
 0.10247804785277326,
 0.13663739713703102,
 0.20495609570554651,
 0.034159349284257755,
 0.034159349284257755,
 0.034159349284257755,
 0.034159349284257755,
 0.034159349284257755,
 0.034159349284257755,
 0.034159349284257755,
 0.06831869856851551,
 0.034159349284257755,
 0.0,
 0.034159349284257755,
 0.034159349284257755,
 0.17079674642128875,
 0.17079674642128875,
 0.034159349284257755,
 0.034159349284257755,
 0.034159349284257755,
 0.06831869856851551,
 0.034159349284257755,
 0.034159349284257755,
 0.06831869856851551,
 0.10247804785277326,
 0.17079674642128875,
 0.06831869856851551,
 0.034159349284257755,
 0.10247804785277326,
 0.034159349284257755,
 0.17079674642128875,
 0.034159349284257755,
 0.034159349284257755,
 0.034159349284257755,
 0.0,
 0.034159349284257755,
 0.06831869856851551,
 0.06831869856851551,
 0.17079674642128875,
 0.034159349284257755,
 0.034159349284257755,
 0.06831869856851551,
 0.23911544498980428,
 0.03415934928425

In [15]:
for vector in vectorBody.values():
    normalize(vector)
for vector in vectorBody2.values():
    normalize(vector)
for vector in vectorBody3.values():
    normalize(vector)
vectorBody3[1]

[0.08801185439477856,
 0.08801185439477856,
 0.10781383133359244,
 0.11375367161471653,
 0.12261736614767482,
 0.08801185439477856,
 0.08801185439477856,
 0.08801185439477856,
 0.08801185439477856,
 0.08801185439477856,
 0.08801185439477856,
 0.08801185439477856,
 0.09996180928400704,
 0.08801185439477856,
 0.0,
 0.08801185439477856,
 0.08801185439477856,
 0.11856261350338372,
 0.11856261350338372,
 0.08801185439477856,
 0.08801185439477856,
 0.08801185439477856,
 0.09996180928400704,
 0.08801185439477856,
 0.08801185439477856,
 0.09996180928400704,
 0.10781383133359244,
 0.11856261350338372,
 0.09996180928400704,
 0.08801185439477856,
 0.10781383133359244,
 0.08801185439477856,
 0.11856261350338372,
 0.08801185439477856,
 0.08801185439477856,
 0.08801185439477856,
 0.0,
 0.08801185439477856,
 0.09996180928400704,
 0.09996180928400704,
 0.11856261350338372,
 0.08801185439477856,
 0.08801185439477856,
 0.09996180928400704,
 0.12613049939001456,
 0.08801185439477856,
 0.08801185439477856

In [16]:
vectorBody[9]

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.3849001794597505,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.3849001794597505,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.19245008972987526,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.19245008972987526,
 0.3849001794597505,
 0.3849001794597505,
 0.19245008972987526,
 0.19245008972987526,
 0.19245008972987526,
 0.19245008972987526,
 0.19245008972987526,
 0.19245008972987526,
 0.19245008972987526,
 0.1

In [17]:
writePath = "D:/Degree of Master/code/CSCI 5523/Project 2/input1.csv"
writePath2 = "D:/Degree of Master/code/CSCI 5523/Project 2/input2.csv"
writePath3 = "D:/Degree of Master/code/CSCI 5523/Project 2/input3.csv"
writeLabel = "D:/Degree of Master/code/CSCI 5523/Project 2/label.csv"
with open(writePath,"w",newline ='') as csvfile: 
    writer = csv.writer(csvfile)
    for i in vectorBody.keys():
        for j in range(0,k): 
            if vectorBody[i][j]<1 and vectorBody[i][j]>0 : writer.writerow((i,j,vectorBody[i][j]))
with open(writePath2,"w",newline ='') as csvfile: 
    writer = csv.writer(csvfile)
    for i in vectorBody2.keys():
        for j in range(0,k): 
            if vectorBody2[i][j]<1 and vectorBody2[i][j]>0: writer.writerow((i,j,vectorBody2[i][j]))
with open(writePath3,"w",newline ='') as csvfile: 
    writer = csv.writer(csvfile)
    for i in vectorBody3.keys():
        for j in range(0,k): 
            if vectorBody3[i][j]<1 and vectorBody3[i][j]>0: writer.writerow((i,j,vectorBody3[i][j]))


In [18]:
writeLabel = "D:/Degree of Master/code/CSCI 5523/Project 2/label.csv"
with open(writeLabel,"w",newline ='') as csvfile: 
    writer = csv.writer(csvfile)
    for tuple in labelList:
        writer.writerow(tuple)

In [66]:
print(len(tokenDimensionNum))

0


In [56]:
tokenD = "D:/Degree of Master/code/CSCI 5523/Project 2/tokenD.csv"
with open(tokenD,"w",newline ='') as csvfile: 
    writer = csv.writer(csvfile)
    for tuple in tokenDimensionNum:
        writer.writerow(tuple)

In [15]:
for p in tokenDimensionNum.keys():
    for k in p:
        if not k.isalnum(): print(False)
print(True)

True


In [18]:
fass=0
for i in tokenDict.values():
    if i>=5: fass+=1
print(fass)

5658
